<a href="https://colab.research.google.com/github/RoaaM/tuning_ML_algorithms/blob/main/method_for_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

# Define the parameter grid
param_grid = {'hidden_layer_sizes': [(10,), (50,), (100,)],
              'activation': ['relu', 'logistic'],
              'learning_rate_init': [0.001, 0.01, 0.1],
              'alpha': [0.0001, 0.001, 0.01]}

# Create the MLP classifier
mlp = MLPClassifier()

# Create the grid search object
grid_search = GridSearchCV(mlp, param_grid, cv=5)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


---

In this example, we define a grid of possible values for the
'hidden_layer_sizes', 'activation', 'learning_rate_init', and 'alpha'
hyperparameters. Then we create an MLPClassifier object and a GridSearchCV
object, passing in the classifier and the parameter grid. The GridSearchCV
object is then fit to the training data, and the best parameters and score are
printed at the end.

---

#Random Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
import numpy as np

# Define the parameter grid
param_dist = {'hidden_layer_sizes': [(10,), (50,), (100,)],
              'activation': ['relu', 'logistic'],
              'learning_rate_init': np.logspace(-4, -1, 4),
              'alpha': np.logspace(-4, -1, 4)}

# Create the MLP classifier
mlp = MLPClassifier()

# Create the random search object
random_search = RandomizedSearchCV(mlp, param_distributions=param_dist, n_iter=10, cv=5)

# Fit the random search to the data
random_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)


---
In this example, we define a distribution of possible values for the
'hidden_layer_sizes', 'activation', 'learning_rate_init', and 'alpha' hyperparameters, then we use RandomizedSearchCV to randomly sample from these
distributions. The 'n_iter' parameter determines how many random combinations
of parameters are tried. The rest of the process is similar to the GridSearchCV example above.

---

#3- BayesSearchCV

In [ ]:
from skopt import BayesSearchCV
from sklearn.neural_network import MLPClassifier

# Define the parameter space
param_space = {'hidden_layer_sizes': [(10,), (50,), (100,)],
              'activation': ['relu', 'logistic'],
              'learning_rate_init': Real(0.0001, 0.1, 'log-uniform'),
              'alpha': Real(0.0001, 0.1, 'log-uniform')}

# Create the MLP classifier
mlp = MLPClassifier()

# Create the Bayesian search object
opt = BayesSearchCV(mlp, param_space, n_iter=10, cv=5)

# Fit the Bayesian search to the data
opt.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters: ", opt.best_params_)
print("Best score: ", opt.best_score_)


---
In this example, we define the parameter space using the same format as in the previous examples, but using Real class to specify the range of the variables and the sampling method. We use the BayesSearchCV class to perform Bayesian optimization, passing in the MLP classifier, the parameter space, the number of iterations and the cross-validation fold. The rest of the process is similar to the other examples above.

---

#4- genetic algorithm

In [ ]:
from deap import base, creator, tools, algorithms
from sklearn.neural_network import MLPClassifier
import numpy as np

# Define the fitness function
def fitness_function(individual):
    mlp = MLPClassifier(hidden_layer_sizes=individual[:3], activation=individual[3],
                        learning_rate_init=individual[4], alpha=individual[5])
    mlp.fit(X_train, y_train)
    return mlp.score(X_test, y_test),

# Define the parameter bounds
param_bounds = [((10, 50, 100),), ('relu', 'logistic'), (0.0001, 0.1), (0.0001, 0.1)]

# Create the genetic algorithm objects
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_bool", np.random.choice, param_bounds)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=6)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_function)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Run the genetic algorithm
pop = toolbox.population(n=50)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, stats=stats, halloffame=hof)

# Print the best parameters and score
print("Best parameters: ", hof[0])
print("Best score: ", fitness_function(hof[0])[0])


---
This is a basic example of how a genetic algorithm can be used to optimize the hyperparameters of a neural network.

---

#5-Particle Swarm Optimization (PSO)

In [ ]:
import numpy as np
import pyswarms as ps
from sklearn.neural_network import MLPClassifier

# Define the fitness function
def fitness_function(x):
    mlp = MLPClassifier(hidden_layer_sizes=x[:, :3], activation=x[:, 3],
                        learning_rate_init=x[:, 4], alpha=x[:, 5])
    mlp.fit(X_train, y_train)
    return -mlp


---
this is an example of how you might use Particle Swarm Optimization (PSO) to tune the hyperparameters of a neural network using the PySwarms library

---

#6- CMA-ES (Covariance Matrix Adaptation Evolution Strategy)

In [ ]:
import numpy as np
import cma
from sklearn.neural_network import MLPClassifier

# Define the fitness function
def fitness_function(x):
    mlp = MLPClassifier(hidden_layer_sizes=x[:3], activation=x[3],
                        learning_rate_init=x[4], alpha=x[5])
    mlp.fit(X_train, y_train)
    return -mlp.score(X_test, y_test)

# Define the initial search point and search space bounds
x0 = np.array([50, 50, 50, 'relu', 0.01, 0.01])
bounds = [(10, 100), (10, 100), (10, 100), ('relu', 'logistic'), (0.0001, 0.1), (0.0001, 0.1)]

# Run the CMA-ES optimization
res = cma.fmin(fitness_function, x0, 0.2, options={'bounds': bounds, 'verbose': -9})

# Print the best parameters and score
print("Best parameters: ", res[0])
print("Best score: ", -res[1])


---
In this example, we use the cma.fmin() function to perform CMA-ES optimization, passing in the fitness function, the initial search point and the search space bounds. The verbose option is set to -9 to reduce the amount of output generated by the algorithm.

---

#7- grid search with early stopping method

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense

# Define the parameter grid
param_grid = {'hidden_layer_sizes': [(10,), (50,), (100,)],
              'activation': ['relu', 'logistic'],
              'learning_rate_init': [0.001, 0.01, 0.1],
              'alpha': [0.0001, 0.001, 0.01]}

# Define the early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

# Define the model
def create_model(hidden_layer_sizes, activation, learning_rate_init, alpha):
    model = Sequential()
    model.add(Dense(hidden_layer_sizes, input_dim=X_train.shape[1], activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create the grid search object
grid_search = GridSearchCV(KerasClassifier(create_model, epochs=100, batch_size=32,
                                           verbose=0, callbacks=[early_stopping]),
                           param_grid, cv=5)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


#8- random search with early stopping method

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

# Define the parameter grid
param_dist = {'hidden_layer_sizes': [(10,), (50,), (100,)],
              'activation': ['relu', 'logistic'],
              'learning_rate_init': np.logspace(-4, -1, 4),
              'alpha': np.logspace(-4, -1, 4)}

# Define the early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

# Define the model
def create_model(hidden_layer_sizes, activation, learning_rate_init, alpha):
    model = Sequential()
    model.add(Dense(hidden_layer_sizes, input_dim=X_train.shape[1], activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create the random search object
random_search = RandomizedSearchCV


#9- Optuna library to perform hyperparameter optimization of a neural network using the Keras library

In [ ]:
import optuna
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.layers import Dense, Dropout
from keras.models import Sequential

def create_model(trial):
    model = Sequential()
    model.add(Dense(trial.suggest_int("units_1", 32, 256), activation=trial.suggest_categorical("activation_1", ["relu", "tanh"]), input_shape=(784,)))
    model.add(Dropout(trial.suggest_uniform("dropout_1", 0.0, 0.5)))
    model.add(Dense(trial.suggest_int("units_2", 32, 256), activation=trial.suggest_categorical("activation_2", ["relu", "tanh"])))
    model.add(Dropout(trial.suggest_uniform("dropout_2", 0.0, 0.5)))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=trial.suggest_categorical("optimizer", ["adam", "sgd"]), loss='categorical_crossentropy', metrics=["accuracy"])
    return model

def objective(trial):
    model = create_model(trial)
    model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_val, y_val))
    val_loss, val_acc = model.evaluate(X_val, y_val)
    return val_loss

study = optuna.create_study()
study.optimize(objective, n_trials=100)
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


---
This is an example of how you might use Optuna to perform hyperparameter optimization of a neural network using the Keras library. In this example, we're using a sequential model with two layers. The number of units, the dropout rate and the activation functions are all set as suggested by Optuna, which uses a Tree-structured Parzen Estimator (TPE) algorithm to optimize the hyperparameters.

---

#10- Keras Tuner to perform hyperparameter tuning of a neural network using the Keras library

In [ ]:
import kerastuner as kt
from keras.datasets import mnist
from keras.layers import Dense
from keras.models import Sequential

(x_train, y_train), (x_val, y_val) = mnist.load_data()

x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_val = x_val.reshape(10000, 784).astype('float32') / 255

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

def model_builder(hp):
    model = Sequential()
    model.add(Dense(hp.Int('units_1', min_value=32, max_value=256, step=32), input_shape=(784,), activation='relu'))
    model.add(Dense(hp.Int('units_2', min_value=32, max_value=256, step=32), activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=hp.Choice('optimizer', ['adam', 'sgd']), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = kt.Hyperband(model_builder, objective='val_acc', max_epochs=10, factor=3, directory='my_dir', project_name='intro_to_kt')

tuner.search(x_train, y_train, validation_data=(x_val, y_val))

best_model = tuner.get_best_models(1)[0]
best_model.summary()


---
In this example, we're using a Sequential model with two layers. The number of units and the optimizer are defined as hyperparameters that are tuned by the Keras Tuner. The Hyperband algorithm is used to perform the tuning.

---

#11- Hyperopt

In [ ]:
from hyperopt import fmin, tpe, hp
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical

(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_val = x_val.reshape(10000, 784).astype('float32') / 255
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

space = {
    'units_1': hp.quniform('units_1', 32, 256, 32),
    'units_2': hp.quniform('units_2', 32, 256, 32),
    'dropout_1': hp.uniform('dropout_1', 0, 0.5),
    'dropout_2': hp.uniform('dropout_2', 0, 0.5),
    'optimizer': hp.choice('optimizer', ['adam', 'sgd']),
}

def train_and_evaluate_model(params):
    model = Sequential()
    model.add(Dense(params['units_1'], input_shape=(784,), activation='relu'))
    model.add(Dropout(params['dropout_1']))
    model.add(Dense(params['units_2'], activation='relu'))
    model.add(Dropout(params['dropout_2']))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=params['optimizer'], loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_val, y_val))
    val_loss, val_acc = model.evaluate(x_val, y_val, verbose=0)
    return {'loss': -val_acc, 'status': 'ok'}

best = fmin(train_and_evaluate_model, space, algo=tpe.suggest, max_evals=50)
print(best)


---
In this example, we're using a Sequential model with two layers. The number of units, the dropout rate and the optimizer are defined as hyperparameters that are optimized by Hyperopt using the Tree-structured Parzen Estimator (TPE) algorithm.

---

#12- autokeras


In [ ]:
!pip install autokeras

In [ ]:
import autokeras as ak
from keras.datasets import mnist

(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_val = x_val.reshape(10000, 784).astype('float32') / 255

clf = ak.ImageClassifier(verbose=True)
clf.fit(x_train, y_train, time_limit=12*60*60)
print(clf.evaluate(x_val, y_val))
